## 3. Die gesammelten Daten bereinigen

Im nächsten Schritt müssen die gesammelten Daten in ein Format gebracht werden, das für neuronale Netzwerke geeignet ist. Genauer wollen wir die JSON-Datei in eine CSV-Datei umwandeln.

Die Tabelle, die wir bekommen wollen, sollte ca. folgendermassen aussehen:

| id | firstname | firstname | yearOfBirth | gender | ... | 32277 | 32219 | 32256 | 32259 | ... |
|-|-|-|-|-|-|-|-|-|-|-|
| "58916" | "Jorgo" | "Ananiadis" | 1969 | 0.0 | ... | 0.0 | 0.25 | 0.25 | 0.75 | ... |
| "55096" | "Jacqueline" | "Badran" | 1961 | 1.0 | ... | 0.0 | 1.0 | 1.0 | 1.0 | ... |
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... |

Die Spalten, die Zahlen als Überschrift haben, enthalten die Antworten der Kandidierenden.

Hier kommt eine neue Bibliothek ins Spiel:
[`pandas`](https://de.wikipedia.org/wiki/Pandas_(Software)) ist eine Programmbibliothek für Python zur Verarbeitung, Analyse und Darstellung von Daten.

In [1]:
import json
import pandas as pd


mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

filename_json = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"
filename_csv_all = "nationalrat_all.csv" if mode == 1 else "ständerat_all.csv"
filename_csv_train = "nationalrat_train.csv" if mode == 1 else "ständerat_train.csv"
filename_csv_test = "nationalrat_test.csv" if mode == 1 else "ständerat_test.csv"

Zuerst laden wir wieder die Rohdaten und schauen uns wieder ein\*e Kandidat*in an.

In [2]:
candidates = json.load(open(filename_json))
candidates[0]

{'id': '56233',
 'firstname': 'Antonio',
 'lastname': 'Abate',
 'yearOfBirth': 1967,
 'profileImageUrl': 'https://backend.smartvote.ch/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsiZGF0YSI6NjI1MjcsInB1ciI6ImJsb2JfaWQifX0=--201173c9e661a50491fc829708435ad0e9f92dd4/Bern-Antonio%20Abate%C3%94%C3%87%C3%B41967.jpg',
 'isIncumbent': False,
 'isElected': False,
 'partyAbbreviation': 'Die Mitte',
 'partyColor': '#D6862B',
 'hasSmartvoteProfile': True,
 'gender': 'm',
 'answers': [{'questionId': '32218', 'value': 75},
  {'questionId': '32261', 'value': 75},
  {'questionId': '32214', 'value': 0},
  {'questionId': '32223', 'value': 0},
  {'questionId': '32224', 'value': 25},
  {'questionId': '32225', 'value': 25},
  {'questionId': '32215', 'value': 75},
  {'questionId': '32226', 'value': 75},
  {'questionId': '32227', 'value': 75},
  {'questionId': '32216', 'value': 100},
  {'questionId': '32217', 'value': 100},
  {'questionId': '32220', 'value': 0},
  {'questionId': '32222', 'value': 25},


Die Antworten auf die Fragen sind aktuell im Format `{'questionId': '32225', 'value': 25}`.
Die `questionId`s sind aus mir unbekannten Gründen scheinbar zufällig angeordnet, lasst uns die also einmal sortieren.

In [3]:
for candidate in candidates:
    candidate["answers"].sort(key=lambda answer: answer["questionId"])
candidates[0]

{'id': '56233',
 'firstname': 'Antonio',
 'lastname': 'Abate',
 'yearOfBirth': 1967,
 'profileImageUrl': 'https://backend.smartvote.ch/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsiZGF0YSI6NjI1MjcsInB1ciI6ImJsb2JfaWQifX0=--201173c9e661a50491fc829708435ad0e9f92dd4/Bern-Antonio%20Abate%C3%94%C3%87%C3%B41967.jpg',
 'isIncumbent': False,
 'isElected': False,
 'partyAbbreviation': 'Die Mitte',
 'partyColor': '#D6862B',
 'hasSmartvoteProfile': True,
 'gender': 'm',
 'answers': [{'questionId': '32214', 'value': 0},
  {'questionId': '32215', 'value': 75},
  {'questionId': '32216', 'value': 100},
  {'questionId': '32217', 'value': 100},
  {'questionId': '32218', 'value': 75},
  {'questionId': '32219', 'value': 25},
  {'questionId': '32220', 'value': 0},
  {'questionId': '32221', 'value': 0},
  {'questionId': '32222', 'value': 25},
  {'questionId': '32223', 'value': 0},
  {'questionId': '32224', 'value': 25},
  {'questionId': '32225', 'value': 25},
  {'questionId': '32226', 'value': 75},
 

Neuronale Netzwerke "mögen" aus mathematischen Gründen normalisierte Zahlen. Heisst: Zahlen zwischen 0 und 1. 
- Für die Antworten ist die Umwandlung trivial, da die `value` Felder bereits alle zwischen 0 und 100 sind.
- Das Netzwerk soll auch das Geschlecht als Input erhalten, also wandeln wir `"m"` und `"f"` in `0` und `1` um. Einige Kandidierenden haben als Geschlecht `"x"` angegeben, das wandeln wir in `0.5` um.

In [4]:
for candidate in candidates:
    for answer in candidate["answers"]:
        answer["value"] = answer["value"] / 100
    if candidate["gender"] == "m":
        candidate["gender"] = 0
        continue
    if candidate["gender"] == "f":
        candidate["gender"] = 1
        continue
    candidate["gender"] = 0.5
candidates[0]

{'id': '56233',
 'firstname': 'Antonio',
 'lastname': 'Abate',
 'yearOfBirth': 1967,
 'profileImageUrl': 'https://backend.smartvote.ch/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsiZGF0YSI6NjI1MjcsInB1ciI6ImJsb2JfaWQifX0=--201173c9e661a50491fc829708435ad0e9f92dd4/Bern-Antonio%20Abate%C3%94%C3%87%C3%B41967.jpg',
 'isIncumbent': False,
 'isElected': False,
 'partyAbbreviation': 'Die Mitte',
 'partyColor': '#D6862B',
 'hasSmartvoteProfile': True,
 'gender': 0,
 'answers': [{'questionId': '32214', 'value': 0.0},
  {'questionId': '32215', 'value': 0.75},
  {'questionId': '32216', 'value': 1.0},
  {'questionId': '32217', 'value': 1.0},
  {'questionId': '32218', 'value': 0.75},
  {'questionId': '32219', 'value': 0.25},
  {'questionId': '32220', 'value': 0.0},
  {'questionId': '32221', 'value': 0.0},
  {'questionId': '32222', 'value': 0.25},
  {'questionId': '32223', 'value': 0.0},
  {'questionId': '32224', 'value': 0.25},
  {'questionId': '32225', 'value': 0.25},
  {'questionId': '32226

Aktuell sind unsere Daten noch hierarchisch verschachtelt. Um die Tabelle erstellen zu können, müssen die Einträge von `'answers'` "herausgezogen" werden.

In [5]:
for candidate in candidates:
    for answer in candidate["answers"]:
        candidate[answer["questionId"]] = answer["value"]
    del candidate["answers"]
candidates[0]

{'id': '56233',
 'firstname': 'Antonio',
 'lastname': 'Abate',
 'yearOfBirth': 1967,
 'profileImageUrl': 'https://backend.smartvote.ch/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsiZGF0YSI6NjI1MjcsInB1ciI6ImJsb2JfaWQifX0=--201173c9e661a50491fc829708435ad0e9f92dd4/Bern-Antonio%20Abate%C3%94%C3%87%C3%B41967.jpg',
 'isIncumbent': False,
 'isElected': False,
 'partyAbbreviation': 'Die Mitte',
 'partyColor': '#D6862B',
 'hasSmartvoteProfile': True,
 'gender': 0,
 '32214': 0.0,
 '32215': 0.75,
 '32216': 1.0,
 '32217': 1.0,
 '32218': 0.75,
 '32219': 0.25,
 '32220': 0.0,
 '32221': 0.0,
 '32222': 0.25,
 '32223': 0.0,
 '32224': 0.25,
 '32225': 0.25,
 '32226': 0.75,
 '32227': 0.75,
 '32228': 0.0,
 '32229': 0.25,
 '32230': 0.0,
 '32231': 0.0,
 '32232': 0.25,
 '32233': 0.25,
 '32234': 0.0,
 '32235': 0.0,
 '32236': 0.75,
 '32237': 0.0,
 '32238': 0.0,
 '32239': 0.75,
 '32240': 0.0,
 '32241': 1.0,
 '32242': 0.25,
 '32243': 1.0,
 '32244': 0.75,
 '32245': 1.0,
 '32246': 0.25,
 '32247': 0.75,
 '322

Beim Analysieren der Daten im 2. Kapitel ist aufgefallen, dass von einigen Parteien nur sehr wenige Personen kandidieren. Lasst uns also alle Kandidierenden aussortieren, die weniger als 9 Kameradinnen und Kameraden haben.

In [6]:
from collections import defaultdict


party_count = defaultdict(int)

# Kandidierende nach Partei aggregieren
for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    if party:
        party_count[party] += 1

# Parteien filtern und Doppelte entfernen
considered_parties = set(
    [party_name for party_name, count in party_count.items() if count >= 10]
)

# Kandidierende filtern
candidates = [
    candidate
    for candidate in candidates
    if candidate["partyAbbreviation"] in considered_parties
]

Als letztes speichern wir die Daten als CSV-Dateien ab.

Dabei unterteilen wir die Kandidierenden per Zufallsprinzip in eine Trainingsmenge und in eine Testmenge, das wird später beim Trainieren und Testen des neuronalen Netzwerks wichtig.

In [7]:
# Hier wird das Dictionary in ein DataFrame umgewandelt, dass einfacher zu handhaben ist
candidates_table = pd.DataFrame(candidates)

candidates_table.to_csv(filename_csv_all)


from sklearn.model_selection import train_test_split


# stratisfy stellt hier sicher, dass die Kandidierenden nach Partei regelmässig aufgeteilt werden
train, test = train_test_split(candidates_table, test_size=0.25, stratify=[candidate["partyAbbreviation"] for candidate in candidates])

train.to_csv(filename_csv_train)
test.to_csv(filename_csv_test)